## Goal

카페데이터 기반으로 가상의 유저수를 설정하고 like 정보를 통해서 rating 데이터를 생성한다.
like 를 할 때 rating 을 10, 안할때를 0으로 해서 정보를 생성

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .config('spark.driver.memory', '15g') \
    .appName('make-rating-data') \
    .getOrCreate()

sc = spark.sparkContext

spark-context 생성.

In [4]:
import os

data_path = os.path.join("..", "data", "data.csv")
raw_data = sc.textFile(data_path)
raw_data_header = raw_data.take(1)[0]

In [5]:
headerList = str(raw_data_header).split(",") # 관리번호 = PK
need_column = ['관리번호', '영업상태구분코드', '영업상태명', '소재지전화',\
        '소재지면적','소재지전체주소', '도로명전체주소', '사업장명', '최종수정시점',\
        '업태구분명', '좌표정보(X)', '좌표정보(Y)', '시설총규모', '홈페이지']
need_column_index = []
column_list = []
for i, value in enumerate(headerList):
    if str(value) in need_column:
        need_column_index.append(i)
        column_list.append(str(value))

In [6]:
data = raw_data.filter(lambda line: line!=raw_data_header) \
    .map(lambda line: line.split(",")) \
    .filter(lambda tokens: tokens[24] == '커피숍' and tokens[7] =='영업/정상') \
    .map(lambda tokens: [tokens[i] for i in need_column_index]) \
    .filter(lambda tokens: tokens[5].split()[0] == '경기도' or tokens[5].split()[0] == '충청북도')

In [8]:
import random
import os, csv

file_name = os.path.abspath('../data/rating.csv')
if not os.path.exists(file_name):
    file = open(file_name, "w", newline="", encoding='utf-8')
    newWriter = csv.writer(file)
    newWriter.writerow(["관리번호", "유저아이디", "좋아요여부"])
    file.close()


file = open(file_name, "a", newline="", encoding='utf-8')
writer = csv.writer(file)

image_user_count = 500
for person in range(1, image_user_count+1):
    for cafe in data.take(1000000):
        # cafe = 가게 정보 line.
        # person 의 확률에 따라서 like 를 추가할것인지 말것인지 결정.
        val = random.random()
        if val > 0.8: # like 정보 갱신.
            row = [cafe[0], person, 10]
            writer.writerow(row)

file.close()

KeyboardInterrupt: 